# Maass form pieces

This is a log of pieces assembled together for the computation of Maass forms.

In [1]:
CC = ComplexField(200)

## Overview

We apply Hejhal's method for the computation of a weight $0$ Maass form on level $1$. This is the simplest possible case. We first implement this in a simple, slow manner.

For a description of the method, refer to my note `202002191140` on Hejhal's algorithm.

## Pullbacks of points to the fundamental domain

Given a point $z \in \mathcal{H}$, we need to compute its image $z^*$ in the fundamental domain. More generally, we will want to track the matrix used (although it has no effect in the weight $0$ no-nebentypus Maass form case). This will later be generalized to fundamental domains of more general congruence subgroups --- but that will build upon this technique.

To do this, we iteratively apply $T: z \mapsto z + 1$ and $S: z \mapsto -1/z$ until the point is within the fundamental domain. Our fundamental domain will be the standard one, where $-1/2 \leq x \leq 1/2$ and $\lvert z \rvert \geq 1$ (where $z = x + iy$).

In [2]:
T = matrix([[1, 1], [0, 1]])
S = matrix([[0, -1], [1, 0]])
Id = matrix([[1, 0], [0, 1]])

def act(gamma, z):
    """
    Computes the action of matrix gamma on point z
    """
    a, b = gamma[0]
    c, d = gamma[1]
    return (a*z + b)/ (c*z + d)

In [3]:
def is_in_fund_domain(z):
    """
    True if z is in fundamental domain, False otherwise.
    """
    x = z.real()
    if x < -1/2 or x > 1/2:
        return False
    if z.norm() < 1:
        return False
    return True


def pullback(z):
    """
    Returns matrix gamma, point z^*, where gamma z = z^* and z^* is in the fundamental domain
    """
    gamma = Id
    while not is_in_fund_domain(z):
        x, y = z.real(), z.imag()
        xshift = -floor(x + .5)
        gamma = T**xshift * gamma
        z = x + xshift + i*y
        if z.norm() < 1:
            z = act(S, z)
            gamma = S * gamma
    return gamma, z

In [4]:
pullback(CC(3.75 + .25*i))

(
[-2  7]
[ 1 -4],

2.0000000000000000000000000000000000000000000000000000000000*I
)

## K-Bessel functions

I require the modified Bessel function of the second kind, denoted by $K_s(x)$. These are implemented in sage as `bessel_K(s, x)`.

In [5]:
bessel_K(CC(1.0), CC(3.0))

0.040156431128194184376705780152684814907243962964308889073382

In [6]:
#def kappa(r, u):
#    return (exp(CC(pi) * CC(r) / 2) * bessel_K(CC(i*r), CC(u)))
def kappa(r, u):
    return bessel_K(CC(i*r), CC(u))

## Setting up the linear system

We will be approximating a Maass form $f$ by its first $M = 50$ (in this case) coefficients,
$$
f(z) \approx \sum_{0 < \lvert n \rvert < 50} a(n) \sqrt{Y} \kappa_{ir}(2 \pi \lvert n \rvert Y) e(nx),
$$
where
$$ \kappa_{ir}(u) = e^{\pi r / 2} K_{ir}(u) $$ is a normalized $K$-bessel function and where $z = x + iY$ for some $Y$.

Let $z_j = x_j + iY = (1/2Q)(j - 1/2) + iY$ where $1 - Q \leq j \leq Q$, where for us $Q = 50$ (in this case). Then summing across these points isolates a single coefficient (indeed, this is Fourier inversion)
$$
a(n) \sqrt{Y} \kappa_{ir}(2 \pi \lvert n \rvert Y)
=
\frac{1}{2Q}
\sum_{j = 1-Q}^Q f(z_j) e(-nx_j).
$$


In [7]:
def make_zj_list(Y, Q=50):
    """
    Returns list of z_j = x_j + iY
    """
    ret = []
    for j in range(1-Q, Q+1):
        ret.append(CC(1/(2*Q)*(j - 0.5) + i*Y))
    return ret

In [8]:
test_zj_list = make_zj_list(0.5, Q=4)
for elem in test_zj_list:
    print(elem)

-0.43750000000000000000000000000000000000000000000000000000000 + 0.50000000000000000000000000000000000000000000000000000000000*I
-0.31250000000000000000000000000000000000000000000000000000000 + 0.50000000000000000000000000000000000000000000000000000000000*I
-0.18750000000000000000000000000000000000000000000000000000000 + 0.50000000000000000000000000000000000000000000000000000000000*I
-0.062500000000000000000000000000000000000000000000000000000000 + 0.50000000000000000000000000000000000000000000000000000000000*I
0.062500000000000000000000000000000000000000000000000000000000 + 0.50000000000000000000000000000000000000000000000000000000000*I
0.18750000000000000000000000000000000000000000000000000000000 + 0.50000000000000000000000000000000000000000000000000000000000*I
0.31250000000000000000000000000000000000000000000000000000000 + 0.50000000000000000000000000000000000000000000000000000000000*I
0.43750000000000000000000000000000000000000000000000000000000 + 0.50000000000000000000000000000000

Let $z_j^* = x_j^* + i y_j^*$ denote the pullback of $z_j$ to the fundamental domain.

In [9]:
def make_zjstar_list(zjlist):
    """
    Returns list of z_j^* in same order as zjlist
    """
    return list(map(lambda zj: pullback(zj)[1], zjlist))

In [10]:
test_zjstar_list = make_zjstar_list(test_zj_list)
print(test_zjstar_list)

[-0.0088495575221238938053097345132743362831858407079646017699114 + 1.1327433628318584070796460176991150442477876106194690265487*I, -0.10112359550561797752808988764044943820224719101123595505618 + 1.4382022471910112359550561797752808988764044943820224719101*I, -0.34246575342465753424657534246575342465753424657534246575342 + 1.7534246575342465753424657534246575342465753424657534246575*I, 0.24615384615384615384615384615384615384615384615384615384615 + 1.9692307692307692307692307692307692307692307692307692307692*I, -0.24615384615384615384615384615384615384615384615384615384615 + 1.9692307692307692307692307692307692307692307692307692307692*I, 0.34246575342465753424657534246575342465753424657534246575342 + 1.7534246575342465753424657534246575342465753424657534246575*I, 0.10112359550561797752808988764044943820224719101123595505618 + 1.4382022471910112359550561797752808988764044943820224719101*I, 0.0088495575221238938053097345132743362831858407079646017699114 + 1.13274336283185840707964601769

In [11]:
def perform_sanity_check_zjstar(zjlist, zjstarlist):
    for zj, zjstar in zip(zjlist, zjstarlist):
        mat, _ = pullback(zj)
        assert (act(mat, CC(zj)) - zjstar).norm() < 0.00000001
    print("Passed.")
perform_sanity_check_zjstar(test_zj_list, test_zjstar_list)

Passed.



As $f$ is weight $0$, we have that $f(z_j) = f(z_j^*)$. Then we also have
$$a(n) \sqrt{Y} \kappa_{ir}(2 \pi \lvert n \rvert Y)
=
\frac{1}{2Q}
\sum_{j = 1-Q}^Q f(z_j^*) e(-nx_j).
$$
To approximate each $f(z_j^*)$, we again represent $f$ by its first $50$ coefficients,
$$
f(z_j^*)
\approx
\sum_{0 < \lvert \ell \rvert \leq 50} a(\ell) \sqrt{y_j^*} \kappa_{ir}(2 \pi \lvert \ell \rvert y_j^*) e(\ell x_j^*).
$$

Inserting this above and collecting, we find that
$$\tag{1}
a(n) \sqrt Y \kappa_{ir}(2 \pi \lvert n \rvert Y)
=
\sum_{0 < \lvert \ell \rvert \leq 50}
a(\ell) V(n, \ell)
$$
where
$$
V(n, \ell) = \frac{1}{2Q}
\sum_{j = 1 - Q}^Q \sqrt{y_j^*}
\kappa_{ir}(2 \pi \lvert \ell \rvert y_j^*)
e(\ell x_j^* - nx_j).
$$

The idea is to consider equation $(1)$ for $\lvert n \rvert \leq 50$, and this system is a linear system. By using the pullbacks $z_j^*$, we hope that this system is non-trivial.

In principle this system remains uncertain since $cf(z)$ is a Maass form with the same eigenvalue for any constant $c$. To get around that, we specify that $a(1) = 1$. **Note that in general this might not always work, since sometimes $a(1)$ could be zero.** But for the initial testing this is sufficient.

**Note also that this normalization convention does not agree with the current system in the LMFDB. The difference is caused by the choice of $K_{ir}$ vs $\kappa_{ir}$.**

In [12]:
def V(n, l, zjlist, r, Q, zjstarlist=None):
    """
    Return the constant V(n, l) --- which also depends on zj, r, and Q --- as
    defined above.
    """
    if zjstarlist is None:
        zjstarlist = make_zjstar_list(zjlist)
    ret = CC(0)
    assert len(zjstarlist) == 2*Q
    for zj, zjs in zip(zjlist, zjstarlist):
        xj = zj.real()
        xjs = zjs.real()
        yjs = zjs.imag()
        ret += CC(sqrt(yjs)) * kappa(r, 2 * pi * abs(l) * yjs) * CC(exp(2*pi*i*(l * xjs - n * xj)))
    ret = ret / (2*Q)
    return ret

In [13]:
V(3, 1, test_zj_list, 9.5**.5, 4)
#V(3, 1, test_zj_list, 9.5**.5, 4) * exp(pi * 9.5**.5 / 2).n()

-0.000012993689505186155312210168421624636000761467477215333142220

### Writing this system as a matrix

For initial sage implementation, we will write this system as a matrix equality $MA = B$ and invert to find the coefficients $A = M^{-1}B$.

Let us describe how this is done.

We gather the elements from equation $(1)$ to one side
$$
0 =
\sum_{0 < \lvert \ell \rvert \leq 50}
a(\ell) V(n, \ell) - a(n) \sqrt Y \kappa_{ir}(2 \pi \lvert n \rvert Y)
= \sum_{0 < \lvert \ell \rvert \leq 50}
a(\ell) C(n, \ell)
$$
where
$$
C(n, \ell) = V(n, \ell) - \delta_{[\ell = n]}\sqrt Y \kappa_{ir}(2 \pi \lvert n \rvert Y).
$$

In [14]:
def make_matrix(size, Y, r):
    Q = size
    zjlist = make_zj_list(Y, Q)
    zjstarlist = make_zjstar_list(zjlist)
    matrix = []
    B = []
    for n in range(-Q, Q+1):
        if n == 0:
            continue
        row = []
        for l in range(-Q, Q+1):
            if l == 0:
                continue
            entry = V(n, l, zjlist, r, Q, zjstarlist=zjstarlist)
            if n == l:
                #entry = entry - sqrt(Y) * bessel_K(i*r, 2 * pi * abs(n) * Y)
                entry = entry - CC(sqrt(Y)) * kappa(r, 2 * pi * abs(n) * Y)
            if l != 1:
                row.append(entry.n())
            else:
                B.append([-entry.n()])
                #if n == 1:
                #    print(V(n, l, zjlist, r, Q, zjstarlist=zjstarlist))
                #    print(sqrt(Y) * kappa(r, 2 * pi * abs(n) * Y))
        matrix.append(row)
    return matrix, B

In [15]:
Q = 8
Y = 0.28
r = 9.53369526135
#testmat, testB = make_matrix(20, 0.5, 3)
testmat, testB = make_matrix(Q, Y, r)

In [16]:
mm = matrix(CC, testmat[1:])
Bm = matrix(CC, testB[1:])

In [17]:
mm.dimensions()

(15, 15)

In [18]:
Bm.dimensions()

(15, 1)

In [19]:
res = mm.inverse() * Bm

In [20]:
indices = [i for i in range(-Q, Q) if i != 0 and i != 1]

In [21]:
for index, re in zip(indices, res):
    print(index, re[0].real())

-8 6.4157544562365146548858143608220917283411576292889316678872e8
-7 0.79398588483254169060190553817159296824648991745527033713374
-6 -0.48869648248955248964144172677800488133893576519410226067621
-5 0.29060778593455862111935858467119788376219040833872730000608
-4 -0.14133144592729883680754136296860334915243101467454409799988
-3 0.45618698845447073224592895792305038640209508835578046500397
-2 1.0683306960901101812070990835770265345429723987682838515171
-1 -0.99999973796300082084985438066206152084904638856675890829703
2 -1.0683309222437650395580746603078205520405730996289936310880
3 -0.45618716040659037490677578064708539434592142993882763497112
4 0.14133135970703934157375597487885141885209669017730925613956
5 -0.29060775341373558097451957193823860527065179074570686719313
6 0.48869644137367958229822274582503068303730522226235968400310
7 -0.79398579222528793246829958189579714083393512823859457916859


In [22]:
def error(res):
    resdict = dict()
    for index, re in zip(indices, res):
        resdict[index] = re[0].real()
    print(resdict[2])
    print(resdict[3])
    print(resdict[5])
error(res)

-1.0683309222437650395580746603078205520405730996289936310880
-0.45618716040659037490677578064708539434592142993882763497112
-0.29060775341373558097451957193823860527065179074570686719313


In [23]:
re[0]

-0.79398579222528793246829958189579714083393512823859457916859 + 3.7266102356198848368739841315777795666405764908791108183318e-62*I

In [24]:
r

9.53369526135000

The column corresponding to $a(1) = 1$ is known, so to see the equation directly we might rewrite this as
$$
-a(1)C(n, 1) = \sum_{\substack{0 < \lvert \ell \rvert \leq 50 \\ \ell \neq 1}} a(\ell) C(n \ell)
$$
Letting $B$ denote the matrix from the $C(n, 1)$, $M$ denote the matrix of the $C(n, \ell)$, and $A$ denote the matrix of the $a(\ell)$ with $\ell \neq 1$, this equation is of the form
$$ B = MA, $$
and thus we solve for $A$ by computing $M^{-1}B$.

There are implementation details concerning the ordering of the rows in the matrix, but the idea is now clear.

## Iteratively improving a solution

We can now produce a list of coefficients for a given $Q$, $Y$, and $r$. To find an actual solution, we will need to solve along a grid, find a candidate spot, and then increase the precision around that point.

In [25]:
def solve_for_coeffs(Q, Y, r):
    mat, B = make_matrix(Q, Y, r)
    mm = matrix(CC, mat[1:])
    Bm = matrix(CC, B[1:])
    res = mm.inverse() * Bm
    indices = [i for i in range(-Q, Q) if i != 0 and i != 1]
    resdict = dict()
    for index, re in zip(indices, res):
        resdict[index] = re[0].real()
    return resdict

In [26]:
coeffdict = solve_for_coeffs(Q, Y, r)

In [27]:
coeffdict

{-8: 6.4157544562365146548858143608220917283411576292889316678872e8,
 -7: 0.79398588483254169060190553817159296824648991745527033713374,
 -6: -0.48869648248955248964144172677800488133893576519410226067621,
 -5: 0.29060778593455862111935858467119788376219040833872730000608,
 -4: -0.14133144592729883680754136296860334915243101467454409799988,
 -3: 0.45618698845447073224592895792305038640209508835578046500397,
 -2: 1.0683306960901101812070990835770265345429723987682838515171,
 -1: -0.99999973796300082084985438066206152084904638856675890829703,
 2: -1.0683309222437650395580746603078205520405730996289936310880,
 3: -0.45618716040659037490677578064708539434592142993882763497112,
 4: 0.14133135970703934157375597487885141885209669017730925613956,
 5: -0.29060775341373558097451957193823860527065179074570686719313,
 6: 0.48869644137367958229822274582503068303730522226235968400310,
 7: -0.79398579222528793246829958189579714083393512823859457916859}

In [28]:
def error_func(r, Y1=0.28, Y2=0.3, Q=8):
    coeffdict1 = solve_for_coeffs(Q, Y1, r)
    coeffdict2 = solve_for_coeffs(Q, Y2, r)
    return ((coeffdict1[2] - coeffdict2[2])**2
            + (coeffdict1[3] - coeffdict2[3])**2 
            + (coeffdict1[5] - coeffdict2[5])**2)

In [29]:
error_func(r)

1.7618177420874112733521873280846517142664220172427439221671e-9

In [30]:
error_func(r - 0.01)

0.41964919464141267087240721520978768203272616705931502702889

In [31]:
def secant_method_single_iteration(r1, r2):
    e1 = error_func(r1)
    e2 = error_func(r2)
    return (r2*e1 - r1*e2)/(e1 - e2)

In [32]:
r1 = 9.51
r2 = 9.52
r3 = secant_method_single_iteration(9.51, 9.52)
print(r3)

9.53137227572494


In [33]:
r4 = secant_method_single_iteration(r2, r3)
print(r4)

9.53209776882166


In [34]:
r5 = secant_method_single_iteration(r3, r4)
print(r5)

9.53282728641699


In [35]:
r6 = secant_method_single_iteration(r4, r5)
print(r6)

9.53316070754963


In [36]:
r7 = secant_method_single_iteration(r5, r6)
print(r7)

9.53337406353245


In [37]:
r8 = secant_method_single_iteration(r6, r7)
print(r8)

9.53349815596829


In [38]:
for idx, rval in enumerate((r1, r2, r3, r4, r5, r6, r7, r8), 1):
    print("Iteration {}:   error {}".format(idx, abs(rval - r)))

Iteration 1:   error 0.0236952613499994
Iteration 2:   error 0.0136952613499997
Iteration 3:   error 0.00232298562505839
Iteration 4:   error 0.00159749252833663
Iteration 5:   error 0.000867974933006721
Iteration 6:   error 0.000534553800370929
Iteration 7:   error 0.000321197817553909
Iteration 8:   error 0.000197105381714024


In [39]:
olderror = 1
for idx, rval in enumerate((r1, r2, r3, r4, r5, r6, r7, r8), 1):
    newerror = abs(rval-r)
    error_ratio = newerror/olderror
    print("Iteration {}:   error_ratio {}".format(idx, error_ratio))
    olderror = newerror

Iteration 1:   error_ratio 0.0236952613499994
Iteration 2:   error_ratio 0.577974690707515
Iteration 3:   error_ratio 0.169619663742923
Iteration 4:   error_ratio 0.687689373151620
Iteration 5:   error_ratio 0.543335832631711
Iteration 6:   error_ratio 0.615863177660213
Iteration 7:   error_ratio 0.600870889573749
Iteration 8:   error_ratio 0.613657288256459


This is expected to continue from abstract numerical analysis. That

## Dragons lie beneath

Below is my true scratchpad. Proceed with utmost caution

In [40]:
r

9.53369526135000

In [41]:
T**2

[1 2]
[0 1]

In [42]:
floor(-2.65)

-3

In [43]:
a = 1 + i

In [44]:
a.is_numeric()

True

In [45]:
a = 1 + 0.2*i

In [46]:
act(S, a)

-0.961538461538461 + 0.192307692307692*I

In [47]:
pi

pi

In [48]:
exp(2 * i * pi)

1

## Testing against a handwritten example

In [49]:
Q = 3
Y = 0.5
r = 3
#testmat, testB = make_matrix(20, 0.5, 9.5336952**.5)
testmat, testB = make_matrix(Q, Y, r)
mm = matrix(testmat[:-1])
Bm = matrix(testB[:-1])

In [50]:
(-2 - 1/2)/6

-5/12

In [51]:
zj_3 = make_zj_list(0.5, 3)

In [52]:
zj_3

[-0.41666666666666662965923251249478198587894439697265625000000 + 0.50000000000000000000000000000000000000000000000000000000000*I,
 -0.25000000000000000000000000000000000000000000000000000000000 + 0.50000000000000000000000000000000000000000000000000000000000*I,
 -0.083333333333333328707404064061847748234868049621582031250000 + 0.50000000000000000000000000000000000000000000000000000000000*I,
 0.083333333333333328707404064061847748234868049621582031250000 + 0.50000000000000000000000000000000000000000000000000000000000*I,
 0.25000000000000000000000000000000000000000000000000000000000 + 0.50000000000000000000000000000000000000000000000000000000000*I,
 0.41666666666666662965923251249478198587894439697265625000000 + 0.50000000000000000000000000000000000000000000000000000000000*I]

In [53]:
for zj in zj_3:
    print(pullback(zj))

([-1 -1]
[ 1  0], -0.016393442622950835425900483796904443668236005421915226691100 + 1.1803278688524591023230935479830939855878536536969874423346*I)
([-1 -1]
[ 1  0], -0.20000000000000000000000000000000000000000000000000000000000 + 1.6000000000000000000000000000000000000000000000000000000000*I)
([ 0 -1]
[ 1  0], 0.32432432432432430729387617448627651144468671439589640575954 + 1.9459459459459459517849567401761335961787100427788811259143*I)
([ 0 -1]
[ 1  0], -0.32432432432432430729387617448627651144468671439589640575954 + 1.9459459459459459517849567401761335961787100427788811259143*I)
([ 1 -1]
[ 1  0], 0.20000000000000000000000000000000000000000000000000000000000 + 1.6000000000000000000000000000000000000000000000000000000000*I)
([ 1 -1]
[ 1  0], 0.016393442622950835425900483796904443668236005421915226691100 + 1.1803278688524591023230935479830939855878536536969874423346*I)


In [54]:
pullback(zj_3[0])

(
[-1 -1]                                                                                                                                  
[ 1  0], -0.016393442622950835425900483796904443668236005421915226691100 + 1.1803278688524591023230935479830939855878536536969874423346*I
)

In [55]:
act(T^-1*S, zj_3[0])

-0.016393442622950835425900483796904443668236005421915226691100 + 1.1803278688524591023230935479830939855878536536969874423346*I

In [56]:
make_zjstar_list(zj_3)

[-0.016393442622950835425900483796904443668236005421915226691100 + 1.1803278688524591023230935479830939855878536536969874423346*I,
 -0.20000000000000000000000000000000000000000000000000000000000 + 1.6000000000000000000000000000000000000000000000000000000000*I,
 0.32432432432432430729387617448627651144468671439589640575954 + 1.9459459459459459517849567401761335961787100427788811259143*I,
 -0.32432432432432430729387617448627651144468671439589640575954 + 1.9459459459459459517849567401761335961787100427788811259143*I,
 0.20000000000000000000000000000000000000000000000000000000000 + 1.6000000000000000000000000000000000000000000000000000000000*I,
 0.016393442622950835425900483796904443668236005421915226691100 + 1.1803278688524591023230935479830939855878536536969874423346*I]

In [57]:
numerical_approx(sqrt(1.18032) * kappa(3, 2*pi*1.18032) * exp(2 * pi * i * ( -0.01639 - (-3)*(-5/12) ) ))

-0.0000170899516493093 - 0.000165364752818340*I

In [58]:
numerical_approx(sqrt(1.6) * kappa(3, 2*pi*1.6) * exp(2 * pi * i * ( -0.2 - (-3)*(-3/12) ) ))

0.0000131599122182715 + 4.27591468041249e-6*I

In [59]:
numerical_approx(sqrt(1.94594) * kappa(3, 2*pi*1.9459) * exp(2 * pi * i * ( 0.32432- (-3)*(-1/12) ) ))

1.51764917621145e-6 + 7.65130897293286e-7*I

In [60]:
numerical_approx(sqrt(1.94594) * kappa(3, 2*pi*1.9459) * exp(2 * pi * i * ( -0.32432- (-3)*(1/12) ) ))

1.51764917621145e-6 - 7.65130897293286e-7*I

In [61]:
numerical_approx(sqrt(1.6) * kappa(3, 2*pi*1.6) * exp(2 * pi * i * ( 0.2 - (-3)*(3/12) ) ))

0.0000131599122182715 - 4.27591468041249e-6*I

In [62]:
numerical_approx(sqrt(1.18032) * kappa(3, 2*pi*1.18032) * exp(2 * pi * i * ( 0.01639 - (-3)*(5/12) ) ))

-0.0000170899516493093 + 0.000165364752818340*I

In [63]:
2*(-.0019024 + .0014649 + 0.0001689)/6

-0.0000895333333333333

In [64]:
V(-3, 1, zj_3, 3, 3)

-8.0521627790273278708641846497788844322476007334751155389176e-7

These agree.

In [65]:
Bm

[   8.05216277902733e-7]
[-0.0000313789019948180]
[ 0.0000549377932823682]
[   0.00561406525439900]
[-0.0000206344504554006]

In [66]:
numerical_approx(sqrt(1.18032) * kappa(3, 2*pi*1.18032) * exp(2 * pi * i * ( -0.01639 - (2)*(-5/12) ) ))

0.0000678820441314201 - 0.000151755052655871*I

In [67]:
numerical_approx(sqrt(1.6) * kappa(3, 2*pi*1.6) * exp(2 * pi * i * ( -0.2 - (2)*(-3/12) ) ))

-4.27591468041249e-6 + 0.0000131599122182715*I

In [68]:
numerical_approx(sqrt(1.94594) * kappa(3, 2*pi*1.9459) * exp(2 * pi * i * ( 0.32432- (2)*(-1/12) ) ))

-1.69688818927829e-6 + 9.62017938293585e-8*I

In [69]:
numerical_approx(sqrt(1.94594) * kappa(3, 2*pi*1.9459) * exp(2 * pi * i * ( -0.32432- (2)*(1/12) ) ))

-1.69688818927829e-6 - 9.62017938293585e-8*I

In [70]:
numerical_approx(sqrt(1.6) * kappa(3, 2*pi*1.6) * exp(2 * pi * i * ( 0.2 - (2)*(3/12) ) ))

-4.27591468041249e-6 - 0.0000131599122182715*I

In [71]:
numerical_approx(sqrt(1.18032) * kappa(3, 2*pi*1.18032) * exp(2 * pi * i * ( 0.01639 - (2)*(5/12) ) ))

0.0000678820441314201 + 0.000151755052655871*I

In [72]:
2*(0.0075564 - 0.000475 - 0.0001888)/6

0.00229753333333333

In [73]:
V(2, 1, zj_3, 3, 3)

0.000020634450455400589886769461998824757738221104770682702795137

In [74]:
sqrt(0.5) * kappa(3, pi)

0.00557309258961281

In [75]:
print(numerical_approx(sqrt(1.18032) * kappa(3, 2*pi*1.18032) * exp(2 * pi * i * ( -0.01639 - (1)*(-5/12) ) )))
print(numerical_approx(sqrt(1.6) * kappa(3, 2*pi*1.6) * exp(2 * pi * i * ( -0.2 - (1)*(-3/12) ) )))
print(numerical_approx(sqrt(1.94594) * kappa(3, 2*pi*1.9459) * exp(2 * pi * i * ( 0.32432- (1)*(-1/12) ) )))
print(numerical_approx(sqrt(1.94594) * kappa(3, 2*pi*1.9459) * exp(2 * pi * i * ( -0.32432- (1)*(1/12) ) )))
print(numerical_approx(sqrt(1.6) * kappa(3, 2*pi*1.6) * exp(2 * pi * i * ( 0.2 - (1)*(3/12) ) )))
print(numerical_approx(sqrt(1.18032) * kappa(3, 2*pi*1.18032) * exp(2 * pi * i * ( 0.01639 - (1)*(5/12) ) )))

-0.000134665101006562 + 0.0000974827086869198*I
0.0000131599122182715 + 4.27591468041249e-6*I
-1.42144738238209e-6 + 9.31757291985001e-7*I
-1.42144738238209e-6 - 9.31757291985001e-7*I
0.0000131599122182715 - 4.27591468041249e-6*I
-0.000134665101006562 - 0.0000974827086869198*I


In [76]:
(-0.01499 + 0.00146 - 0.0001)*2/6

-0.00454333333333333

In [77]:
V(1, 1, zj_3, 3, 3)

-0.000040972664786193975048908458421967887381102430997560701505075

In [78]:
0.620384 + 0.00456

0.624944000000000

In [79]:
Bm

[   8.05216277902733e-7]
[-0.0000313789019948180]
[ 0.0000549377932823682]
[   0.00561406525439900]
[-0.0000206344504554006]

In [80]:
for n in range(-3, 3):
    if n == 0:
        continue
    if n == 1:
        print(sqrt(0.5)*kappa(3, pi) - V(1, 1, zj_3, 3, 3))
        continue
    print(-V(n, 1, zj_3, 3, 3))

8.0521627790273278708641846497788844322476007334751155389176e-7
-0.000031378901994818004058548814816573132427093818296336187805905
0.000054937793282368243093733375486743926615606826936923833954761
0.00561406525439901
-0.000020634450455400589886769461998824757738221104770682702795137


The construction of B appears to be correct.

In [81]:
print(numerical_approx(sqrt(1.18032) * kappa(3, 2*pi*2*1.18032) * exp(2 * pi * i * ( 2*(-0.01639) - (2)*(-5/12) ) )))
print(numerical_approx(sqrt(1.6) * kappa(3, 2*pi*2*1.6) * exp(2 * pi * i * ( 2*(-0.2) - (2)*(-3/12) ) )))
print(numerical_approx(sqrt(1.94594) * kappa(3, 2*pi*2*1.9459) * exp(2 * pi * i * ( 2*0.32432- (2)*(-1/12) ) )))
print(numerical_approx(sqrt(1.94594) * kappa(3, 2*pi*2*1.9459) * exp(2 * pi * i * ( 2*(-0.32432)- (2)*(1/12) ) )))
print(numerical_approx(sqrt(1.6) * kappa(3, 2*pi*2*1.6) * exp(2 * pi * i * ( 2*0.2 - (2)*(3/12) ) )))
print(numerical_approx(sqrt(1.18032) * kappa(3, 2*pi*2*1.18032) * exp(2 * pi * i * ( 2*0.01639 - (2)*(5/12) ) )))

2.95066791798738e-8 - 8.97492980023567e-8*I
4.23356756340555e-10 + 3.07586687999816e-10*I
2.81163373028717e-12 - 6.46310647776134e-12*I
2.81163373028717e-12 + 6.46310647776134e-12*I
4.23356756340555e-10 - 3.07586687999816e-10*I
2.95066791798738e-8 + 8.97492980023567e-8*I


In [82]:
sum([numerical_approx(sqrt(1.18032) * kappa(3, 2*pi*2*1.18032) * exp(2 * pi * i * ( 2*(-0.01639) - (2)*(-5/12) ) )),
numerical_approx(sqrt(1.6) * kappa(3, 2*pi*2*1.6) * exp(2 * pi * i * ( 2*(-0.2) - (2)*(-3/12) ) )),
numerical_approx(sqrt(1.94594) * kappa(3, 2*pi*2*1.9459) * exp(2 * pi * i * ( 2*0.32432- (2)*(-1/12) ) )),
numerical_approx(sqrt(1.94594) * kappa(3, 2*pi*2*1.9459) * exp(2 * pi * i * ( 2*(-0.32432)- (2)*(1/12) ) )),
numerical_approx(sqrt(1.6) * kappa(3, 2*pi*2*1.6) * exp(2 * pi * i * ( 2*0.2 - (2)*(3/12) ) )),
numerical_approx(sqrt(1.18032) * kappa(3, 2*pi*2*1.18032) * exp(2 * pi * i * ( 2*0.01639 - (2)*(5/12) ) ))])/6

9.97761585664821e-9

In [83]:
V(2, 2, zj_3, 3, 3)

9.9753681592983991485396160351763832420881374883237208574905e-9

A random middle element seems to agree as well. Look at the incredibly small sizes, though. Perhaps I'm suffering only from precision problems.

In [84]:
Mmat = matrix(testmat)

In [85]:
Mmat.dimensions()

(6, 5)

In [86]:
for val in Mmat[0]:
    print(val)

-0.0000145545497841146
6.34043997137191e-9
8.05216277902810e-7
-6.34043997137186e-9
-5.19661533216426e-12


In [87]:
expected = [
    V(-3, -3, zj_3, 3, 3) - sqrt(0.5)*kappa(3, 3*pi),
    V(-3, -2, zj_3, 3, 3),
    V(-3, -1, zj_3, 3, 3),
    V(-3, 2, zj_3, 3, 3),
    V(-3, 3, zj_3, 3, 3)]
for val in expected:
    print(val)

-0.0000145545497841146
6.3404399713719064855947143842245940283640042765099400762678e-9
8.0521627790280964167248803436123283632157134877355129119988e-7
-6.3404399713718634833430597910500828073567892090290637413122e-9
-5.1966153321642559296935066519685596058586617774822688588438e-12


In [88]:
for val in Mmat[2]:
    print(val)

-1.14944143550645e-11
-2.33706142032866e-8
-0.00561406525439900
-3.00186408626583e-8
-1.66793875489469e-11


In [89]:
expected = [
    V(-1, -3, zj_3, 3, 3),
    V(-1, -2, zj_3, 3, 3),
    V(-1, -1, zj_3, 3, 3) - sqrt(0.5)*kappa(3, pi),
    V(-1, 2, zj_3, 3, 3),
    V(-1, 3, zj_3, 3, 3)]
for val in expected:
    print(val)

-1.1494414355064505333058780087661134002735421057807958944388e-11
-2.3370614203286595754516827777518521175201803524808063831140e-8
-0.00561406525439901
-3.0018640862658300083633757428033970104209810314242174728140e-8
-1.6679387548946899791640973740158287124958086230343469806929e-11


In [90]:
for val in Mmat[3]:
    print(val)

-1.66793875489469e-11
-3.00186408626583e-8
-0.0000549377932823682
-2.33706142032866e-8
-1.14944143550645e-11


In [91]:
expected = [
    V(1, -3, zj_3, 3, 3),
    V(1, -2, zj_3, 3, 3),
    V(1, -1, zj_3, 3, 3),
    V(1, 2, zj_3, 3, 3),
    V(1, 3, zj_3, 3, 3)]
for val in expected:
    print(val)

-1.6679387548946899791640973740158287124958086230343469806929e-11
-3.0018640862658300083633757428033970104209810314242174728140e-8
-0.000054937793282368243093733375486743926615606826936923833954761
-2.3370614203286595754516827777518521175201803524808063831140e-8
-1.1494414355064505333058780087661134002735421057807958944388e-11


In [92]:
for val in Mmat[4]:
    print(val)

1.26354644114724e-11
2.11283780321047e-8
0.0000313789019948180
-0.000329440225214611
3.64139006416494e-12


In [93]:
expected = [
    V(2, -3, zj_3, 3, 3),
    V(2, -2, zj_3, 3, 3),
    V(2, -1, zj_3, 3, 3),
    V(2, 2, zj_3, 3, 3) - sqrt(0.5)*kappa(3, 2*pi),
    V(2, 3, zj_3, 3, 3)]
for val in expected:
    print(val)

1.2635464411472388525840874530622333726317595203903594501264e-11
2.1128378032104695755554864568307559582215501706840639647851e-8
0.000031378901994818004058548814816573132427093818296336187805905
-0.000329440225214611
3.6413900641649397074444530946161812974323172282078016505231e-12


In [94]:
known_r = 9.53369526135
known_r

9.53369526135000

In [95]:
z = CC(1/2 + 1/3 * i)

In [96]:
z

0.50000000000000000000000000000000000000000000000000000000000 + 0.33333333333333333333333333333333333333333333333333333333333*I

In [97]:
z.real()

0.50000000000000000000000000000000000000000000000000000000000

In [98]:
z.imag()

0.33333333333333333333333333333333333333333333333333333333333

In [99]:
def make_single_matrix(size, Y, r, withtime=True):
    if withtime:
        now = time.time()
    Q = size
    zjlist = make_zj_list(Y, Q)
    zjstarlist = make_zjstar_list(zjlist)
    if withtime:
        print("lists made in {}".format(time.time() - now))
    matrix = []
    B = []
    for n in range(-Q, Q+1):
        if withtime:
            now = time.time()
        if n == 0:
            continue
        row = []
        for l in range(-Q, Q+1):
            if l == 0:
                continue
            entry = CDF(V(n, l, zjlist, CDF(r), CDF(Q), zjstarlist=zjstarlist))
            if n == l:
                #entry = entry - sqrt(Y) * bessel_K(i*r, 2 * pi * abs(n) * Y)
                entry = entry - CDF(sqrt(Y)) * kappa(r, 2 * pi * abs(n) * Y)
            if l != 1:
                row.append(entry.n())
            else:
                B.append([-entry.n()])
                #if n == 1:
                #    print(V(n, l, zjlist, r, Q, zjstarlist=zjstarlist))
                #    print(sqrt(Y) * kappa(r, 2 * pi * abs(n) * Y))
        matrix.append(row)
        if withtime:
            print("row {} computed in {} seconds".format(n, time.time() - now))
    return matrix, B

In [100]:
def make_single_matrix(size, Y, r):
    Q = size
    zjlist = make_zj_list(Y, Q)
    zjstarlist = make_zjstar_list(zjlist)
    matrix = []
    for n in range(-Q, Q+1):
        if n == 0:
            continue
        row = []
        for l in range(-Q, Q+1):
            if l == 0:
                continue
            entry = CDF(V(n, l, zjlist, CDF(r), CDF(Q), zjstarlist=zjstarlist) / CDF(sqrt(Y)) * kappa(r, 2 * pi * abs(n) * Y))
            if n == l:
                #entry = entry - sqrt(Y) * bessel_K(i*r, 2 * pi * abs(n) * Y)
                entry = entry - 1
            row.append(entry.n())
        matrix.append(row)
    return matrix

In [101]:
Q = 8
Y = 0.28
r = 9.53369526135
#testmat, testB = make_matrix(20, 0.5, 3)
testmat = make_single_matrix(Q, Y, r)

In [102]:
testmat

[[-1.00000000000000,
  6.76250716914684e-30,
  2.28721444244064e-26,
  -1.34664167710065e-23,
  4.77828566419976e-21,
  -1.05951473852331e-18,
  8.32892274376649e-17,
  1.81029235120464e-16 + 4.40011885056943e-77*I,
  -1.81029235120464e-16 - 4.40011885056943e-77*I,
  -8.32892274376649e-17,
  1.05951473852331e-18,
  -4.77828566419976e-21,
  1.34664167710065e-23,
  -2.28721444244064e-26,
  -6.76250716914684e-30,
  2.39474633814077e-31],
 [1.22384321982340e-30,
  -1.00000000000000,
  -4.67802750356432e-26,
  4.17718550728856e-23,
  -1.69565097761505e-20,
  4.18722556049326e-18 - 2.04332696722026e-80*I,
  -3.94661903910044e-16 + 8.36946725773420e-77*I,
  -1.53470498343845e-15 + 3.34778690309368e-76*I,
  3.04115656527753e-16 + 3.34778690309368e-76*I,
  2.45005593743221e-16,
  -3.71850231614453e-18,
  1.86649471321958e-20,
  -5.86882590655565e-23,
  1.23898710250029e-25,
  -1.02573759428308e-28,
  -5.63152563550636e-31],
 [-4.73752736218752e-30,
  8.72695564693753e-28,
  -1.00000000000000,
 

In [103]:
len(testmat)

16

In [104]:
len(testmat[0])

16

In [105]:
testsagematrix = Matrix(testmat)

In [106]:
potans = testsagematrix.rref()[0]

In [107]:
for entry in potans:
    print(entry)

1.00000000000000
0.000000000000000
0.000000000000000
0.000000000000000
0.000000000000000
0.000000000000000
1.02219103766563e-107*I
0.000000000000000
0.000000000000000
0.000000000000000
0.000000000000000
0.000000000000000
0.000000000000000
0.000000000000000
0.000000000000000
0.000000000000000
